In [1]:
professions_category_tokens = {
    'Медик' : [
      'лекарство',
      'больница',
      'боль',
      'пациент',
      'нежелательные',
      'явления',
      'лечение',
      'доктор',
      'врач',
      'проблемы',
      'здоровье',
      'лечение',
      'болезнь',
      'лекарство',
      'аптека',
      'таблетка',
      'иньекция',
    ],
    'Government workers' : [
    ],
    'Полицейский' : [
      'должностные',
      'должность',
      'пост',
      'инструкции',
      'документ',
      'процессуальное',
      'решение',
      'жалоба',
      'правопорядок',
      'охрана',
      'противодействие',
      'полиция',
      'борьба',
      'преступление',
      'кодекс',
    ],
    'In office' : [
    ],
    'Films, tv and radio' : [
    ],
    'Arts and culture' : [
    ],
    'Бизнес сфера' : [
      'деньги',
      'доход',
      'расход',
      'экономика',
      'доллары',
      'валюта',
      'актив',
      'договор',
      'кредит',
      'бизнес',
      'работники',
      'увольнение',
      'найм',
      'отдел',
      'экономисты',
      'менеджеры',
      'маркетинг'
    ],
    'Science' : [
    ],
    'Education in school' : [
    ],
    'Education in university' : [
    ],
    'Сфера услуг' : [
      'оплата',
      'услуг',
      'заказ',
      'выполнение',
      'обязательства',
      'доставка',
      'невыполнение',
      'организация',
      'работа',
      'клиент',
      'задача',
      'услуга',
    ],
    'Дизайн' : [
      'логотип',
      'креатив',
      'рисование',
      'макет',
      'дизайн',
      'создание'
    ],
    'Программирование' : [
      'разработчик',
      'проект',
      'код',
      'программа',
      'команда',
      'система',
      'программирование',
      'проект'
    ]
}

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
def forms(s):
    s = str(s).lower()
    res = []
    if s[-1]=='а':  # 1 склонение
        res.append(s)  # И
        res.append(s[:-1]+'ы')  # Р
        res.append(s[:-1]+'е')  # Д
        res.append(s[:-1]+'у')  # В
        res.append(s[:-1]+'ой')  # Т
        res.append(s[:-1]+'е')  # П
    if s[-1]=='я':  # 1 склонение
        res.append(s)  # И
        res.append(s[:-1]+'и')  # Р
        res.append(s[:-1]+'е')  # Д
        res.append(s[:-1]+'ю')  # В
        res.append(s[:-1]+'ей')  # Т
        res.append(s[:-1]+'е')  # П
    elif s[-1]=='о':  # 2 склонение (с окончанием)
        res.append(s)  # И
        res.append(s[:-1]+'а')  # Р
        res.append(s[:-1]+'у')  # Д
        res.append(s)  # В
        res.append(s[:-1]+'ом')  # Т
        res.append(s[:-1]+'е')  # П
    elif s[-1]=='е':  # 2 склонение (с окончанием)
        res.append(s)  # И
        res.append(s[:-1]+'я')  # Р
        res.append(s[:-1]+'ю')  # Д
        res.append(s)  # В
        res.append(s[:-1]+'ем')  # Т
        res.append(s[:-1]+'и')  # П
    elif s[-1]=='ь':  # 3 склонение
        res.append(s)  # И
        res.append(s[:-1]+'и')  # Р
        res.append(s[:-1]+'и')  # Д
        res.append(s[:-1]+'и')  # В
        res.append(s[:-1]+'ью')  # Т
        res.append(s[:-1]+'и')  # П
    else:  # если хз, какое склонение, добавляет сразу несколько
           # лучше, чем ничего
        res.append(s)
        res.append(s[:-1]+'и')
        res.append(s[:-1]+'е')
        res.append(s[:-1]+'а')
        res.append(s[:-1]+'у')
        res.append(s[:-1]+'ы')
    return list(set(res))

In [57]:
def category_percent(token_vocabulary):
    # создаем базу тег - професиия
    X = []  # тег
    Y = []  # профессия
    for proff, tag in professions_category_tokens.items():
        X += tag
        for i in range(len(tag)):
            Y.append(proff)
    
    # создаем базу падежи и теги профессии - профессия
    Xp = []
    Yp = []
    for i in range(len(X)) :
        f = forms(X[i])
        Xp += f
        Yp += [Y[i]] * len(f)
    
    vectorizer = CountVectorizer()  # векторизируем текст Xp
    Xp = vectorizer.fit_transform(Xp)
    
    clf = LogisticRegression()  # обучаем нейросеть
    clf.fit(Xp,Yp)
    
    # сортируем пары ключ - значение по максимальному значению
    keys = []  
    values = []
    for key, value in token_vocabulary.items():
        keys.append(key)
        values.append(int(value*100))
    values_sorted = sorted(values, reverse=True)

    sum_values = 0
    keys_sorted = []
    values_sorted_persent = []
    for i in range(len(values_sorted)):
        sum_values += values_sorted[i]
        keys_sorted.append(keys[values.index(values_sorted[i])])
    for i in range(len(values_sorted)):
        values_sorted_persent.append (int(values_sorted[i] * 100/sum_values))
        
    keys_sorted_searched = []  # в эту переменную кладутся профессии от нейронки
    for i in range(len(keys_sorted)):
        s = keys_sorted[i]
        s = (s.lower().split())
        s = vectorizer.transform(s).toarray()
        keys_sorted_searched.append(clf.predict(s)[0])
        
    dict_i = {}
    for i in range(len(keys_sorted_searched)):
        dict_i.update({keys_sorted_searched[i] : values_sorted_persent[i]})
    return dict_i

In [16]:
token_vocabulary = {
    'лечение' : 0.4,
    'заказ' : 0.3,
    'разработка' : 0.7
}

In [58]:
print(category_percent(token_vocabulary))

{'Бизнес сфера': 50, 'Медик': 28, 'Сфера услуг': 21}


/home/alexandr/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/alexandr/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
